<h1>Ford Care Guesser</h1>

Modelo desenvolvido a fim de ver se os componentes do trem de força correm risco ou não

## Data Import

## Model Training

In [ ]:
import pandas as pd
pd.set_option('display.max_columns',None)

from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pickle

In [ ]:
df = pd.read_csv("../data/ford_guesser_fuel.csv", index_col='Unnamed: 0')

In [8]:
y_pred = clf.predict(x_test)

In [9]:
print("Acurácia: ",metrics.accuracy_score(y_test, y_pred))

Acurácia:  0.705904161646121


In [10]:
df['$ALERT'] = clf.predict(x)

In [11]:
df['$ALERT'].value_counts()

False    32589
True      3372
Name: $ALERT, dtype: int64

In [13]:
def get_train_data():
    df = pd.read_csv("../data/ford_guesser_fuel.csv", index_col='Unnamed: 0')
    return df

def format_guesser_data(df):
    
    df = df.join(pd.get_dummies(df.MARK))
    df = df.join(pd.get_dummies(df.MODEL))
    df.drop(['MARK','MODEL','VEHICLE_ID','ERROR CODE'],inplace=True,axis=1)
    df.AUTOMATIC = df.AUTOMATIC.replace({'s':1, 'n':0})
    df.DTC_NUMBER = df.DTC_NUMBER.replace({'MIL is OFF1 codes':1, 'MIL is OFF0 codes':0})
    df['SHORT TERM FUEL TRIM BANK 1'].fillna(df['SHORT TERM FUEL TRIM BANK 1'].mean(), inplace=True)

    for c in df.columns:
        if df[c].isna().sum() > 0:
            df[c] = df[c].fillna(df[c].mean())
    return df

def train_guesser(df, model):
    x = df[['AIR_INTAKE_TEMP', 'CAR_YEAR', 'DAYS_OF_WEEK',
           'DTC_NUMBER', 'ENGINE_COOLANT_TEMP', 'ENGINE_LOAD', 'ENGINE_POWER',
           'ENGINE_RPM', 'HOURS', 'INTAKE_MANIFOLD_PRESSURE','MIN',
            'MONTHS', 'SHORT TERM FUEL TRIM BANK 1', 'SPEED',
           'THROTTLE_POS', 'TIME', 'TIMING_ADVANCE', 'YEAR']]

    y = df['ALERT']

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

    
    return model.fit(x_train, y_train)

def export_guesser_pickle(clf):
    filename = '../app/ford_care/ford_guesser.sav'
    pickle.dump(clf, open(filename, 'wb'))
    
if __name__ == "__main__":
    df = get_train_data()
    df = format_guesser_data(df)
    clf = RandomForestClassifier(n_estimators=100)
    clf_trained = train_guesser(df, clf)
    

## Model Metrics

## Export model